## Machine Learning Project: Customer Churn Predictions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Loading and displaying data from Telco Customer Churn (https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

In [2]:
data_file = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(data_file.head())

# Check column names and data types
print(data_file.info())

# Summary statistics
print(data_file.describe())

# Check for missing values
print(data_file.isnull().sum())

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract Pape

### Convert categorical columns into numbers
Using LabelEncoder

In [3]:
x = data_file.drop("Churn", axis=1)  # Drop the 'Churn' column from features (X)
y = data_file["Churn"]  # 'Churn' column is the target variable (y)
gender = pd.get_dummies(data_file['gender'], drop_first=True)  # Convert categorical columns into numbers
encoder = LabelEncoder()
y=encoder.fit_transform(y)
categorical_columns = ["gender", "Partner", "Dependents", "PhoneService","OnlineSecurity","DeviceProtection",
                      "TechSupport","StreamingTV","StreamingMovies","PaperlessBilling","MultipleLines","Contract",
                      "PaymentMethod","OnlineBackup"]  # Add all categorical column names here

for col in categorical_columns:
    x[col] = encoder.fit_transform(x[col])  # Encode all categorical columns
x["TotalCharges"] = pd.to_numeric(x["TotalCharges"], errors="coerce")

x = x.drop("customerID", axis=1)
x["TotalCharges"].fillna(0, inplace=True)

x = pd.get_dummies(x, columns=["InternetService", "Contract", "PaymentMethod"], drop_first=True).astype(int)

### Set train and test variables and implement the Logistic Regression model
Adapting the maximum iterations depinding on the computing capacity.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Check accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.82


### Report on the accuracy and precision of the model

In [5]:
# Get confusion matrix
cm = confusion_matrix(y_test, model.predict(X_test))
print("Confusion Matrix:\n", cm)

# Get classification report (precision, recall, F1-score)
report = classification_report(y_test, model.predict(X_test))
print("Classification Report:\n", report)


Confusion Matrix:
 [[1529  168]
 [ 261  367]]
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.90      0.88      1697
           1       0.69      0.58      0.63       628

    accuracy                           0.82      2325
   macro avg       0.77      0.74      0.75      2325
weighted avg       0.81      0.82      0.81      2325



### Inject the Predicted Churn into the inital data set

In [6]:
y_pred_mapped = np.where(y_pred == 0, "No", "Yes")
data_file.loc[X_test.index, 'PredictedChurn'] = y_pred_mapped
print(data_file)

      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL             No  ...   
1        